# PreImcal Version

In [1]:
import preimcal
from joblib import Parallel, delayed

Welcome to eht-imaging! v 1.2.2 



In [2]:
import ehtim as eh
import numpy as np
import matplotlib.pyplot as plt
import ehtim.scattering.stochastic_optics as so
import sys
import os

def precal(obs,scan_avg=True, tint = 60.0, is_normalized=False, syserr=0.02):
    if scan_avg:
        print("Here")
        obscal = obsproc = preimcal.preim_pipeline(obs, is_normalized=is_normalized, 
                                                   syserr=syserr, do_psd_noise=False, 
                                                   ref_optype='quarter1', 
                                                   do_deblurr=True, 
                                                   ref_scale=1.0)
        
        obscal.add_scans(dt=2*60.0/3600.)
        obscal = obscal.avg_coherent(60.0)
        obscal = obscal.avg_coherent(0., scan_avg=True)
        return obscal.copy().avg_coherent(0.0, scan_avg=True)
    else:
        obscal = obsproc = preimcal.preim_pipeline(obs, is_normalized=is_normalized, 
                                                   tint=tint, syserr=syserr, 
                                                   do_psd_noise=False, 
                                                   ref_optype='quarter1',
                                                   do_deblurr=True, 
                                                   ref_scale=1.0)
        
        return obscal




def makefile_preimcal(uvfits_path, scan_avg=True, tint = 60.0, is_normalized=False, syserr=0.02):
    if scan_avg:
        averaging_modes = ['scan-average']
    else:
        averaging_modes = ["snapshot%02d"%tint]
    #noise_fracs = [0.02, 0.05]
    noise_fracs = [syserr]

    for averaging_mode in averaging_modes:
        for noise_frac in noise_fracs:
            # Set up paths
            simn = os.path.splitext(os.path.basename(uvfits_path))[0]
            obs_filename = uvfits_path.split('/')[-1].split('.uvf')[0]
            obs_dir = '/'.join(uvfits_path.split('/')[:-1])+"/" + simn
            out_dir = obs_dir + '/preimcal_pipeline/%s/noisefrac%s/'%(averaging_mode, noise_frac)
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
    
            # Load data
            obs = eh.obsdata.load_uvfits(uvfits_path)
            print(len(obs.tlist()))
    
            # Divide into scans
            print(averaging_mode)
            obs_proc = precal(obs, scan_avg=scan_avg, is_normalized=is_normalized, syserr=noise_frac, tint=tint).copy()
            # Add noise, collect scan info, and export scans
            if scan_avg:
                obs_scans = obs_proc.split_obs(scan_gather=True)
            else:
                obs_scans = obs_proc.split_obs()
            nscans = len(obs_scans)
            print("Nscans: ", nscans)
            tstart = []
            tstop = []
            sites = []
            gs = []
            isc = -1
            for scan_number in range(nscans):
                obs_scan = obs_scans[scan_number]
                print(len(obs_scan.data))


                if len(obs_scan.data["time"]) < 3:
                    print("Skipping scan", scan_number)
                    continue
                isc += 1
                out_file = out_dir + obs_filename + '_preprocessed_%s_noisefrac%s_scan%s.uvfits'%(averaging_mode, noise_frac, isc)
                obs_scan.save_uvfits(out_file)

                gs.append(scan_number)
    
                tstart.append(obs_scan.data['time'][0])
                tstop.append(obs_scan.data['time'][-1])

                scan_sites = []
                for site in obs_scan.tarr['site']:
                    if site in obs_scan.data['t1'] or site in obs_scan.data['t2']:
                        scan_sites.append(site)
                sites.append(scan_sites)
    
    
            # Write out scan info
            if noise_frac == noise_fracs[0]:
                with open(obs_dir + '/preimcal_pipeline/%s/scaninfo_%s.txt'%(averaging_mode, averaging_mode), 'w') as f:
                    print('# Scan number, tstart, tstop, sites', end='\n', file=f)
                    for i in range(len(sites)):
                        print('%03d %.5f %.5f %s'%(i, tstart[i], tstop[i], ','.join(sites[i])), end='\n', file=f)
                f.close()
    
    
    
def makefile_preimcal_camp(uvfits_path, scan_avg=True, tint = 60.0, is_normalized=False, syserr=0.02):
    if scan_avg:
        averaging_modes = ['scan-average']
    else:
        averaging_modes = ["snapshot%02d"%tint]
    #noise_fracs = [0.02, 0.05]
    noise_fracs = [syserr]

    for averaging_mode in averaging_modes:
        for noise_frac in noise_fracs:
            # Set up paths
            simn = os.path.splitext(os.path.basename(uvfits_path))[0]
            obs_filename = uvfits_path.split('/')[-1].split('.uvf')[0]
            obs_dir = '/'.join(uvfits_path.split('/')[:-1])+"/" + simn
            out_dir = obs_dir + '/preimcal_pipeline_camp/%s/noisefrac%s/'%(averaging_mode, noise_frac)
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
    
            # Load data
            obs = eh.obsdata.load_uvfits(uvfits_path)
            print(len(obs.tlist()))
    
            # Divide into scans
            print(averaging_mode)
            obs_proc = precal(obs, scan_avg=scan_avg, is_normalized=is_normalized, syserr=noise_frac, tint=tint).copy()
            # Add noise, collect scan info, and export scans
            if scan_avg:
                obs_scans = obs_proc.split_obs(scan_gather=True)
            else:
                obs_scans = obs_proc.split_obs()
            nscans = len(obs_scans)
            print("Nscans: ", nscans)
            tstart = []
            tstop = []
            sites = []
            gs = []
            isc = -1
            for scan_number in range(nscans):
                obs_scan = obs_scans[scan_number]
                print(len(obs_scan.data))


                if len(obs_scan.data["time"]) < 4:
                    print("Skipping scan", scan_number)
                    continue
                isc += 1
                out_file = out_dir + obs_filename + '_preprocessed_%s_noisefrac%s_scan%s.uvfits'%(averaging_mode, noise_frac, isc)
                obs_scan.save_uvfits(out_file)

                gs.append(scan_number)
    
                tstart.append(obs_scan.data['time'][0])
                tstop.append(obs_scan.data['time'][-1])

                scan_sites = []
                for site in obs_scan.tarr['site']:
                    if site in obs_scan.data['t1'] or site in obs_scan.data['t2']:
                        scan_sites.append(site)
                sites.append(scan_sites)
    
    
            # Write out scan info
            if noise_frac == noise_fracs[0]:
                with open(obs_dir + '/preimcal_pipeline_camp/%s/scaninfo_%s.txt'%(averaging_mode, averaging_mode), 'w') as f:
                    print('# Scan number, tstart, tstop, sites', end='\n', file=f)
                    for i in range(len(sites)):
                        print('%03d %.5f %.5f %s'%(i, tstart[i], tstop[i], ','.join(sites[i])), end='\n', file=f)
                f.close()
    
    
    
    
    


# ScanAvg

In [22]:
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal)(
        "dataCal/uvfits_MoD_precal_stage1/dataset%03d.uvfits"%i, 
        is_normalized=True, tint=60.0, scan_avg=True
        ) 
    for i in range(0,35)
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   17.7s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   27.3s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:   44.7s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   55.1s
[Parallel(n_jobs=4)]: Done  32 out of  35 | elapsed:  1.2min remaining:    6.8s
[Parallel(n_jobs=4)]: Done  35 out of  35 | elapsed:  1.3min finished


In [17]:
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal_camp)(
        "dataCal/uvfits_MoD_precal_stage2/dataset%03d.uvfits"%i, 
        is_normalized=True, tint=60.0, scan_avg=True
        ) 
    for i in range(35,119)
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   13.8s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   21.6s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:   35.5s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   43.3s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  1.9min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done  84 out of  84 | elapsed:  2.5min finished


# Snapshot

In [23]:
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal)(
        "dataCal/uvfits_MoD_precal_stage1/dataset%03d.uvfits"%i, 
        is_normalized=True, tint=60.0, scan_avg=False
        ) 
    for i in range(0,35)
)


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   45.2s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  2.1min
[Parallel(n_jobs=4)]: Done  32 out of  35 | elapsed:  3.0min remaining:   17.0s
[Parallel(n_jobs=4)]: Done  35 out of  35 | elapsed:  3.5min finished


In [24]:
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal)(
        "dataCal/uvfits_MoD_precal_stage1/dataset%03d.uvfits"%i, 
        is_normalized=True, tint=120.0, scan_avg=False
        ) 
    for i in range(0,35)
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   33.1s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   49.5s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done  32 out of  35 | elapsed:  2.3min remaining:   12.7s
[Parallel(n_jobs=4)]: Done  35 out of  35 | elapsed:  2.5min finished


In [ ]:
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal)(
        "dataCal/uvfits_MoD_precal_stage1/dataset%03d.uvfits"%i, 
        is_normalized=True, tint=180.0, scan_avg=False
        ) 
    for i in range(0,35)
)

In [20]:
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal_camp)(
        "dataCal/uvfits_MoD_precal_stage2/dataset%03d.uvfits"%i, 
        is_normalized=True, tint=60.0, scan_avg=False
        ) 
    for i in range(35,119)
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   33.7s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   52.5s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  2.7min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  3.3min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  4.2min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  5.0min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:  6.3min
[Parallel(n_jobs=4)]: Done  84 out of  84 | elapsed:  6.7min finished


In [21]:
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal_camp)(
        "dataCal/uvfits_MoD_precal_stage2/dataset%03d.uvfits"%i, 
        is_normalized=True, tint=120.0, scan_avg=False
        ) 
    for i in range(35,119)
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   24.5s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   38.6s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  3.5min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:  4.3min
[Parallel(n_jobs=4)]: Done  84 out of  84 | elapsed:  4.6min finished


# SgrA

In [13]:
makefile_preimcal_camp(
        "SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits", 
        is_normalized=True, tint=60.0, scan_avg=True
        ) 

Loading uvfits:  SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)
/home/ptiede/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


2061
scan-average
Here
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.
Pre-imaging processing: deblurring data


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


Splitting Observation File into 43 times
Nscans:  43
3
Skipping scan 0
3
Skipping scan 1
6
Building NX table
6
Building NX table
3
Skipping scan 4
6
Building NX table
6
Building NX table
6
Building NX table
4
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
15
Building NX table
15
Building NX table
10
Building NX table
21
Building NX table
10
Building NX table
15
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
10
Building NX table
21
Building NX table
21
Building NX table
15
Building NX table
15
Building NX table
10
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
3
Skipping scan 42


In [3]:
makefile_preimcal_camp(
        "SgrAData/SgrA_ER6_ver2021-04-15/3598/hops/hops_3598_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits", 
        is_normalized=True, tint=60.0, scan_avg=True
        ) 
makefile_preimcal_camp(
        "SgrAData/SgrA_ER6_ver2021-04-15/3598/hops/hops_3598_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits", 
        is_normalized=True, tint=60.0, scan_avg=False
        ) 
makefile_preimcal_camp(
        "SgrAData/SgrA_ER6_ver2021-04-15/3598/hops/hops_3598_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits", 
        is_normalized=True, tint=120.0, scan_avg=False
        ) 

Loading uvfits:  SgrAData/SgrA_ER6_ver2021-04-15/3598/hops/hops_3598_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)
/home/ptiede/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


1152
scan-average
Here
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.
Pre-imaging processing: deblurring data


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


Splitting Observation File into 25 times
Nscans:  25
3
Skipping scan 0
3
Skipping scan 1
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
15
Building NX table
15
Building NX table
15
Building NX table
21
Building NX table
21
Building NX table
15
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
15
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
15
Building NX table
Loading uvfits:  SgrAData/SgrA_ER6_ver2021-04-15/3598/hops/hops_3598_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
1152
snapshot60
Pre-imaging processing: time-averaging.
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.
Pre-imaging

/home/ptiede/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Splitting Observation File into 193 times
Nscans:  193
3
Skipping scan 0
3
Skipping scan 1
1
Skipping scan 2
1
Skipping scan 3
1
Skipping scan 4
1
Skipping scan 5
1
Skipping scan 6
1
Skipping scan 7
3
Skipping scan 8
3
Skipping scan 9
3
Skipping scan 10
3
Skipping scan 11
3
Skipping scan 12
3
Skipping scan 13
1
Skipping scan 14
1
Skipping scan 15
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX t

Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
Loading uvfits:  SgrAData/SgrA_ER6_ver2021-04-15/3598/hops/hops_3598_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
1152
snapshot120
Pre-imaging processing: time-averaging.
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.
Pre-imaging processing: deblurring data
Splitting Observation File into 110 times
Nscans:  110
3
Skipping scan 0
1
Skipping scan 1
1
Skipping scan 2
1
Skipping scan 3
3
Skipping scan 4
3
Skipping scan 5
3
Skipping scan 6
3
Skipping scan 7
1
Skipping scan 8
6
Building NX table
No NX ta

In [4]:
makefile_preimcal_camp(
        "SgrAData/SgrA_ER6_ver2021-04-15/3598/hops/hops_3598_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits", 
        is_normalized=True, tint=240.0, scan_avg=False
        ) 
makefile_preimcal_camp(
        "SgrAData/SgrA_ER6_ver2021-04-15/3598/hops/hops_3598_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits", 
        is_normalized=True, tint=300.0, scan_avg=False
        ) 
makefile_preimcal_camp(
        "SgrAData/SgrA_ER6_ver2021-04-15/3598/hops/hops_3598_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits", 
        is_normalized=True, tint=360.0, scan_avg=False
        ) 

Loading uvfits:  SgrAData/SgrA_ER6_ver2021-04-15/3598/hops/hops_3598_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
1152
snapshot240
Pre-imaging processing: time-averaging.
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.
Pre-imaging processing: deblurring data
Splitting Observation File into 67 times
Nscans:  67
3
Skipping scan 0
1
Skipping scan 1
3
Skipping scan 2
3
Skipping scan 3
3
Skipping scan 4
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Buildin

Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
10
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No

In [14]:
makefile_preimcal_camp(
        "SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits", 
        is_normalized=True, tint=60.0, scan_avg=False
        ) 

Loading uvfits:  SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
2061
snapshot60
Pre-imaging processing: time-averaging.
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.


/home/ptiede/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Pre-imaging processing: deblurring data
Splitting Observation File into 350 times
Nscans:  350
3
Skipping scan 0
3
Skipping scan 1
3
Skipping scan 2
3
Skipping scan 3
3
Skipping scan 4
3
Skipping scan 5
3
Skipping scan 6
3
Skipping scan 7
3
Skipping scan 8
3
Skipping scan 9
3
Skipping scan 10
3
Skipping scan 11
3
Skipping scan 12
3
Skipping scan 13
3
Skipping scan 14
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 17
3
Skipping scan 18
3
Skipping scan 19
3
Skipping scan 20
3
Skipping scan 21
3
Skipping scan 22
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 26
3
Skipping scan 27
3
Skipping scan 28
3
Skipping scan 29
3
Skipping scan 30
3
Skipping scan 31
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 33
3
Skipping scan 34
3
Skipping scan 35
3
Skipping scan 36
3
Skipping scan 37
3
Skipping scan 38


Building NX table
No NX table in saved uvfits
10
Building NX table
No NX table in saved uvfits
10
Building NX table
No NX table in saved uvfits
10
Building NX table
No NX table in saved uvfits
10
Building NX table
No NX table in saved uvfits
10
Building NX table
No NX table in saved uvfits
9
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX ta

In [15]:
makefile_preimcal_camp(
        "SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits", 
        is_normalized=True, tint=120.0, scan_avg=False
        ) 

Loading uvfits:  SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
2061
snapshot120
Pre-imaging processing: time-averaging.
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.
Pre-imaging processing: deblurring data
Splitting Observation File into 195 times
Nscans:  195
3
Skipping scan 0
3
Skipping scan 1
3
Skipping scan 2
3
Skipping scan 3
3
Skipping scan 4
3
Skipping scan 5
3
Skipping scan 6
3
Skipping scan 7
3
Skipping scan 8
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 11
3
Skipping scan 12
3
Skipping scan 13
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 16
3
Skipping scan 17
3
Skipping scan 18
6
Building NX

6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 192
3
Skipping scan 193
3
Skipping scan 194


In [4]:
makefile_preimcal_camp(
        "SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits", 
        is_normalized=True, tint=180.0, scan_avg=False
        ) 

Loading uvfits:  SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


2061
snapshot180
Pre-imaging processing: time-averaging.
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.


/home/ptiede/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Pre-imaging processing: deblurring data
Splitting Observation File into 149 times
Nscans:  149
3
Skipping scan 0
3
Skipping scan 1
3
Skipping scan 2
3
Skipping scan 3
3
Skipping scan 4
3
Skipping scan 5
3
Skipping scan 6
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 8
3
Skipping scan 9
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 12
3
Skipping scan 13
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 15
3
Skipping scan 16
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 18
3
Skipping scan 19
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 22
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 24
3
Skipping scan 25
10
Building NX table
No NX table in saved uvfits
3
Skipping scan 27
3
Skipping scan 28
3
Skipping scan 29
10
Building NX table
No NX table in saved uvfits
10
Building NX table
No NX table 

In [5]:
makefile_preimcal_camp(
        "SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits", 
        is_normalized=True, tint=240.0, scan_avg=False
        ) 

Loading uvfits:  SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
2061
snapshot240
Pre-imaging processing: time-averaging.
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.
Pre-imaging processing: deblurring data
Splitting Observation File into 118 times
Nscans:  118
3
Skipping scan 0
3
Skipping scan 1
3
Skipping scan 2
3
Skipping scan 3
3
Skipping scan 4
3
Skipping scan 5
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 7
3
Skipping scan 8
6
Building NX table
No NX table in saved uvfits
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 11
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 13
3
Skipping scan 14
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 16
6
Building NX table
N

In [6]:
makefile_preimcal_camp(
        "SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits", 
        is_normalized=True, tint=300.0, scan_avg=False
        ) 

Loading uvfits:  SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_normalized_10s.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
2061
snapshot300
Pre-imaging processing: time-averaging.
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.
Pre-imaging processing: deblurring data
Splitting Observation File into 101 times
Nscans:  101
3
Skipping scan 0
3
Skipping scan 1
3
Skipping scan 2
3
Skipping scan 3
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 5
3
Skipping scan 6
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 8
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 10
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 12
6
Building NX table
No NX table in saved uvfits
3
Skipping scan 14
6
Building NX table
No NX table in saved uvfits
3
Skipping

# Best Times

In [3]:
makefile_preimcal_camp(
        "SgrAData/BestTimes/hops_3598_SGRA_LO_netcal_LMTcal_10s_regionIII.uvfits", 
        is_normalized=False, tint=60.0, scan_avg=False
        ) 
makefile_preimcal_camp(
        "SgrAData/BestTimes/hops_3599_SGRA_LO_netcal_LMTcal_10s_regionIII.uvfits", 
        is_normalized=False, tint=60.0, scan_avg=False
        ) 

Loading uvfits:  SgrAData/BestTimes/hops_3598_SGRA_LO_netcal_LMTcal_10s_regionIII.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


341
snapshot60
Pre-imaging processing: Obtaining the light curve from the intra-site baselines
Pre-imaging processing: Normalizing the total flux density
Pre-imaging processing: time-averaging.
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.
Pre-imaging processing: deblurring data


/home/ptiede/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Pre-imaging processing: rescaling the total flux density.
Splitting Observation File into 58 times
Nscans:  58
9
Building NX table
No NX table in saved uvfits
10
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
15
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Building NX table
No NX table in saved uvfits
21
Build

In [4]:
import glob

# Validation

In [7]:
files = glob.glob("ValidationData/variabledatasets_inoisy_v4/ehtim_allepoch/uvfits_synthetic_3599/thermal+gain+scattering/*netcal_LMTcal_normalized*.uvfits")

In [8]:
print(files)

['ValidationData/variabledatasets_inoisy_v4/ehtim_allepoch/uvfits_synthetic_3599/thermal+gain+scattering/crescent_hops_hi_netcal_LMTcal_normalized.uvfits', 'ValidationData/variabledatasets_inoisy_v4/ehtim_allepoch/uvfits_synthetic_3599/thermal+gain+scattering/simple_disk_hops_hi_netcal_LMTcal_normalized.uvfits', 'ValidationData/variabledatasets_inoisy_v4/ehtim_allepoch/uvfits_synthetic_3599/thermal+gain+scattering/grmhd_hops_hi_netcal_LMTcal_normalized.uvfits', 'ValidationData/variabledatasets_inoisy_v4/ehtim_allepoch/uvfits_synthetic_3599/thermal+gain+scattering/ring_hops_lo_netcal_LMTcal_normalized.uvfits', 'ValidationData/variabledatasets_inoisy_v4/ehtim_allepoch/uvfits_synthetic_3599/thermal+gain+scattering/ring+hs_period30_hops_lo_netcal_LMTcal_normalized.uvfits', 'ValidationData/variabledatasets_inoisy_v4/ehtim_allepoch/uvfits_synthetic_3599/thermal+gain+scattering/ring+hs_period30_hops_hi_netcal_LMTcal_normalized.uvfits', 'ValidationData/variabledatasets_inoisy_v4/ehtim_allepoch

In [9]:
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal_camp)(
        f, 
        is_normalized=True, tint=60.0, scan_avg=True
        ) 
    for f in files
)
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal_camp)(
        f, 
        is_normalized=True, tint=60.0, scan_avg=False
        ) 
    for f in files
)
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal_camp)(
        f, 
        is_normalized=True, tint=120.0, scan_avg=False
        ) 
    for f in files
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   18.9s
[Parallel(n_jobs=4)]: Done  11 out of  16 | elapsed:   29.1s remaining:   13.2s
[Parallel(n_jobs=4)]: Done  13 out of  16 | elapsed:   37.1s remaining:    8.6s
[Parallel(n_jobs=4)]: Done  16 out of  16 | elapsed:   39.3s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   35.1s
[Parallel(n_jobs=4)]: Done  11 out of  16 | elapsed:   53.7s remaining:   24.4s
[Parallel(n_jobs=4)]: Done  13 out of  16 | elapsed:  1.2min remaining:   16.2s
[Parallel(n_jobs=4)]: Done  16 out of  16 | elapsed:  1.2min finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   21.4s
[Parallel(n_jobs=4)]: Done  11 out of  16 | elapsed:   33.2s remaining:   15.1s
[Parallel(n_jobs=4)]: Done  13 out of  16 | elapsed:

In [10]:
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal_camp)(
        f, 
        is_normalized=True, tint=180.0, scan_avg=False
        ) 
    for f in files
)
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal_camp)(
        f, 
        is_normalized=True, tint=240.0, scan_avg=False
        ) 
    for f in files
)
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal_camp)(
        f, 
        is_normalized=True, tint=300.0, scan_avg=False
        ) 
    for f in files
)
r = Parallel(n_jobs=4, verbose=10)(
    delayed(makefile_preimcal_camp)(
        f, 
        is_normalized=True, tint=360.0, scan_avg=False
        ) 
    for f in files
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   18.3s
[Parallel(n_jobs=4)]: Done  11 out of  16 | elapsed:   28.3s remaining:   12.8s
[Parallel(n_jobs=4)]: Done  13 out of  16 | elapsed:   36.6s remaining:    8.4s
[Parallel(n_jobs=4)]: Done  16 out of  16 | elapsed:   37.9s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   15.1s
[Parallel(n_jobs=4)]: Done  11 out of  16 | elapsed:   23.5s remaining:   10.7s
[Parallel(n_jobs=4)]: Done  13 out of  16 | elapsed:   30.3s remaining:    7.0s
[Parallel(n_jobs=4)]: Done  16 out of  16 | elapsed:   31.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   14.1s
[Parallel(n_jobs=4)]: Done  11 out of  16 | elapsed:   23.8s remaining:   10.8s
[Parallel(n_jobs=4)]: Done  13 out of  16 | elapsed: